In [ ]:
# Prepare Database as Backend
import sqlite3

# Create a new SQLite database
conn = sqlite3.connect('data/university_chatbot.db')
cursor = conn.cursor()

# Create tables
# Students Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS students (
        matriculation_number INTEGER PRIMARY KEY,
        name TEXT NOT NULL,
        surname TEXT NOT NULL,
        address TEXT
    )
''')

# Courses Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS courses (
        course_id INTEGER PRIMARY KEY,
        course_name TEXT NOT NULL,
        instructor TEXT NOT NULL
    )
''')

# Registrations Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS registrations (
        registration_id INTEGER PRIMARY KEY,
        matriculation_number INTEGER NOT NULL,
        course_id INTEGER NOT NULL,
        FOREIGN KEY (matriculation_number) REFERENCES students (matriculation_number),
        FOREIGN KEY (course_id) REFERENCES courses (course_id)
    )
''')

# Exam Results Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS exam_results (
        result_id INTEGER PRIMARY KEY,
        matriculation_number INTEGER NOT NULL,
        course_id INTEGER NOT NULL,
        grade TEXT NOT NULL,
        FOREIGN KEY (matriculation_number) REFERENCES students (matriculation_number),
        FOREIGN KEY (course_id) REFERENCES courses (course_id)
    )
''')

# Commit changes and close the connection
conn.commit()
conn.close()


In [ ]:
# Populate Database with data

import random

# Connect to the SQLite database
conn = sqlite3.connect('data/university_chatbot.db')
cursor = conn.cursor()

# Sample data for students
students = [
    (201001, 'Alice', 'Smith', 'Jakobstraße 123'),
    (201002, 'Bob', 'Johnson', 'Schlägelstraße 456'),
    (201003, 'Carol', 'Williams', 'Borsigplatz 789'),
    (201004, 'David', 'Brown', 'Kamener Straße 101'),
    (201005, 'Eve', 'Davis', 'Priorstraße 202'),
    (201006, 'Frank', 'Miller', 'Burgholzstraße 303'),
    (201007, 'Grace', 'Wilson', 'Clausthaler Straße 404'),
    (201008, 'Henry', 'Moore', 'Nordstraße 505'),
    (201009, 'Ivy', 'Taylor', 'Am Waldfried 606'),
    (201010, 'Jack', 'Anderson', 'Westfalenhüttenallee 707')
]

# Sample data for courses
courses = [
    (101, 'Introduction to Computer Science', 'Dr. John Doe'),
    (102, 'Advanced Mathematics', 'Dr. Jane Smith'),
    (103, 'Physics for Engineers', 'Dr. Emily Johnson')
]

# Insert students into the database
cursor.executemany('INSERT INTO students VALUES (?,?,?,?)', students)

# Insert courses into the database
cursor.executemany('INSERT INTO courses VALUES (?,?,?)', courses)

# Generate registrations (2 for each student)
registrations = []
for student in students:
    registered_courses = random.sample(courses, 2)  # Randomly pick 2 courses for each student
    for course in registered_courses:
        registrations.append((None, student[0], course[0]))

# Insert registrations into the database
cursor.executemany('INSERT INTO registrations (registration_id, matriculation_number, course_id) VALUES (?,?,?)', registrations)

# Generate exam results (1 for each student)
exam_results = []
for student in students:
    course_id = random.choice(registrations)[2]  # Randomly pick one course from registrations
    grade = random.choice(['A', 'B', 'C', 'D', 'E', 'F'])  # Randomly assign a grade
    exam_results.append((None, student[0], course_id, grade))

# Insert exam results into the database
cursor.executemany('INSERT INTO exam_results (result_id, matriculation_number, course_id, grade) VALUES (?,?,?,?)', exam_results)

# Commit changes and close the connection
conn.commit()
conn.close()

In [ ]:
#Importing nltk + packages
import nltk

nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

def process_input(user_input):
    tokens = word_tokenize(user_input)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    return words

# Example usage
sample_input = "I would like to change my address because I have moved out."
processed_input = process_input(sample_input)
print(processed_input)


In [ ]:
def identify_intent(processed_input):
    intent_keywords = {
        'register_exam': ['register', 'exam'],
        'change_address': ['change', 'address'],
        'change_surname': ['change', 'surname'],
        # Add more intents and associated keywords here for Query
    }

    for intent, keywords in intent_keywords.items():
        if all(keyword in processed_input for keyword in keywords):
            return intent

    return 'unknown_intent'

# Example usage
intent = identify_intent(processed_input)


In [ ]:
import re
def has_postcode(user_input):
    postcode_pattern = re.compile(r'\b\d{5}\b')
    return bool(re.search(postcode_pattern, user_input))
def identify_yes_no_answer(input):
    intent_keywords = {
        'yes': ['true', 'yes','yeah', 'yep'],
        'no': ['wrong', 'false', 'no', 'not', 'nope', 'nah'],
        # Add more intents and associated keywords here
    }

    for intent, keywords in intent_keywords.items():
        if any(keyword in input for keyword in keywords):
                return intent

    return 'unknown_intent'
if intent == 'change_address':
    #Ask for student's matriculation number
    matriculation_number = input("Please enter your matriculation number: ")
    #Connect to the SQLite database
    conn = sqlite3.connect('data/university_chatbot.db')
    cursor = conn.cursor()
    #Check if student exists
    cursor.execute('SELECT * FROM students WHERE matriculation_number=?', (matriculation_number,))
    student = cursor.fetchone()
    if student is None:
        print("Sorry, you are not registered as a student.")
    else:
        #Ask for new address
        new_address = input("Please enter your new address ")
        while not has_postcode(new_address):
            postal_code = input("Can you give me your postal code please? ")
            new_address = new_address + " " + postal_code
        
        answer = input("Let me summerize once again: You have moved out and your new address is " + new_address + "? ")
        processed_answer = identify_yes_no_answer(answer)
        while processed_answer == 'unknown_intent':
            answer = input("Sorry, I didn't understand. Please answer with yes or no. ")
            processed_answer = identify_yes_no_answer(answer)
        if processed_answer == 'no':
            new_address = input("Please enter your new address ")
            while not has_postcode(new_address):
                postal_code = input("Can you give me your postal code please? ")
                new_address = new_address + " " + postal_code

        #Update student's address
        cursor.execute('UPDATE students SET address=? WHERE matriculation_number=?', (new_address, matriculation_number))
        conn.commit()
        print("Your address has been updated.")
    #Close the connection
    conn.close()
    

In [ ]:
import re
import sqlite3


def identify_yes_no_answer(input):
    intent_keywords = {
        'yes': ['true', 'yes', 'yeah', 'yep'],
        'no': ['wrong', 'false', 'no', 'not', 'nope', 'nah'],
        # Add more intents and associated keywords here
    }

    for intent, keywords in intent_keywords.items():
        if any(keyword in input for keyword in keywords):
            return intent

    return 'unknown_intent'

def change_surname():
    # Ask for student's matriculation number
    matriculation_number = input("Please enter your matriculation number: ")
    
    # Connect to the SQLite database
    conn = sqlite3.connect('data/university_chatbot.db')
    cursor = conn.cursor()
    
    # Check if student exists
    cursor.execute('SELECT * FROM students WHERE matriculation_number=?', (matriculation_number,))
    student = cursor.fetchone()
    
    if student is None:
        print("Sorry, you are not registered as a student.")
    else:
        # Ask for new surname
        new_surname = input("Please enter your new surname : ")
         # Confirm the updated surname
        answer = input("Let me summarize once again: Your new surname is " + new_surname + "? ")
        processed_answer = identify_yes_no_answer(answer)
        
        while processed_answer == 'unknown_intent':
            answer = input("Sorry, I didn't understand. Please answer with yes or no. ")
            processed_answer = identify_yes_no_answer(answer)
        
        if processed_answer == 'no':
            new_surname = input("Please enter your correct new surname: ")
            # Update student's surname again
            cursor.execute('UPDATE students SET surname=? WHERE matriculation_number=?', (new_surname, matriculation_number))
            conn.commit()
        
        print("Your surname has been updated.")

        # Update student's surname
        cursor.execute('UPDATE students SET surname=? WHERE matriculation_number=?', (new_surname, matriculation_number))
        conn.commit()
        
    
    # Close the connection
    conn.close()


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

def process_input(user_input):
    tokens = word_tokenize(user_input)
    tokens = [w.lower() for w in tokens]
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    words = [word for word in stripped if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    return words

# Example usage
sample_input = "I would like to status for the Machine Learning exam."
processed_input = process_input(sample_input)


In [ ]:
import sqlite3
#Register methods to query exam status and grade
def query_exam_status(matriculation_number, course_id):
    conn = sqlite3.connect('data/university_chatbot.db')
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM registrations WHERE matriculation_number=? AND course_id=?', (matriculation_number, course_id))
    registration = cursor.fetchone()
    conn.close()
    if registration:
        return "Registered"
    else:
        return "Not Registered or Examination Completed"

def query_exam_grade(matriculation_number, course_id):
    conn = sqlite3.connect('data/university_chatbot.db')
    cursor = conn.cursor()
    cursor.execute('SELECT grade FROM exam_results WHERE matriculation_number=? AND course_id=?', (matriculation_number, course_id))
    result = cursor.fetchone()
    conn.close()
    if result:
        return result[0]
    else:
        return "No grade available or Examination not yet passed"
print(query_exam_grade(201003, 101))

print(query_exam_status(201001, 101))

In [ ]:
import re

# Example function for information extraction
def extract_information(input_text):
    # Regex patterns for different entities
    matric_number_pattern = r'\b\d{5,7}\b'
    course_id_pattern = r'\b\d{3}\b'
    address_pattern = r'\b\d{1,5}\s\w+\s\w+\b'  # Simplified pattern for addresses like '123 Main Street'

    # Extract entities
    matric_number = re.findall(matric_number_pattern, input_text)
    course_ids = re.findall(course_id_pattern, input_text)
    address = re.findall(address_pattern, input_text)

    # Determine possible intentions based on keywords
    possible_intentions = []
    if 'grade' in input_text.lower():
        possible_intentions.append('query_exam_grade')
    if 'status' in input_text.lower():
        possible_intentions.append('query_exam_status')
    if 'change' in input_text.lower() and 'surname' in input_text.lower():
        possible_intentions.append('change_surname')
   
    return {
        'matriculation_number': matric_number[0] if matric_number else None,
        'intentions': possible_intentions,
        'course_id': course_ids[0] if course_ids else None,
        'address': address[0] if address else None,
        # 'surname_old': ...,
        # 'surname_new': ...,
    }

# Mapping of intents to functions and required variables
intent_to_function = {
    'query_exam_grade': {'function': query_exam_grade, 'required_vars': ['matriculation_number', 'course_id']},
    'query_exam_status': {'function': query_exam_status, 'required_vars': ['matriculation_number', 'course_id']},
    'change_surname': {'function': change_surname, 'required_vars': ['matriculation_number',]}
    # TODO
}

# Example chatbot logic
def chatbot_logic(input_text):
    extracted_info = extract_information(input_text)
    print(extracted_info)

    if len(extracted_info['intentions']) > 1:
        # Feedback loop for multiple intentions
        # Ask user to specify the correct intent
        return "I found multiple possible intents: " + ", ".join(extracted_info['intentions']) + ". Please specify."

    elif len(extracted_info['intentions']) == 1:
        intent = extracted_info['intentions'][0]
        print(intent)
        required_vars = intent_to_function[intent]['required_vars']
               # Extract the required variables for the intent
        args = [extracted_info[var] for var in required_vars if var in extracted_info]

        # Check if all required information is available
        if all(arg is not None for arg in args):
            # Call the function associated with the intent and pass the required arguments
            return intent_to_function[intent]['function'](*args)
        else:
            # Ask for missing information
            missing_vars = [var for var in required_vars if not extracted_info.get(var)]
            return f"Please provide your {' and '.join(missing_vars)} for {intent}."

    else:
        return "I'm not sure what you're asking for. Could you please clarify?"

# Example usage
response = chatbot_logic("I want to query the status for course 101. My matriculation number is 201234.")
print(response)

In [ ]:
def chatLoop():
    print("Welcome to the PacMan Chatbot! Type 'quit to exit'")
    while True:
        # User inputs their query
        user_input = input("You: ")
        print(user_input)
        # Check if the user wants to quit the chat
        if user_input.lower() == 'quit':
            print("Chatbot: Goodbye!")
            break
        processed_input = process_input(user_input)
        
        response = chatbot_logic(user_input)

        print(f"Chatbot: {response}")
chatLoop()